# 対抗馬手法　kmeans++
    kmeans++アルゴリズムで各FPGAの周波数をクラスタリング

- XPの論文では，経年劣化したFPGAは周波数の分布が異なると予測
- 実際にHSPICEを用いたシュミレーションでも，経年劣化したFPGAは周波数が低い&多峰性
- クラスタ数が他と異なるところを異常値として判定する

# 手法
    kmeans++  シルエット値

## [kmeans++]
1. 中心点の初期値を確率的に決定
2. データポイントと中心点とのユークリッド平方距離を計算
3. 中心点との距離が最も近いクラスターに分類
4. すべての点(クラスター内)の平均を計算して，中心点を割り当て
5. 再び，新しい中心点とのユークリッド平方距離を求め，距離が近いクラスターに分類
6. 3,4,5を繰り返し，中心点の位置を移動

### 中心点の初期値を確率的に決定
1. データ点のうち一つをランダムに選択し，その座標をクラスタ1の中心点の初期値とする
2. 入力される全てのデータ点と，すでに座標が決定している中心点のうち最も近いものとのユークリッド距離を計算
3. いずれかのデータ点を次の中心点とする，
    - その際のデータ点X(p)の選ばれる確率
    $$
    \dfrac {d\left( x_{p}\right) ^{2}}{\sum ^{n-1}_{i=0}d\left( x_{i}\right) ^{2}}
    $$
    - 最近某クラスタから遠ければ遠いほど，選ばれる確率は高くなる
    - すでに中心点として選ばれているデータ点との距離は0なので，分母の総和にも影響を与えないし，選ばれる確率も0となる
4. 手順2-3をk個の中心点が決定するまで繰り返す

- kmeans++を使うことで，計算速度(実行速度)が上がり，適切にクラスタできる確率も上がる
- 極値も避けることができる

### kmeansの問題点
- kmeansアルゴリズムはクラスタ数を人の手で決定しなければならない

## [シルエット値]
    シルエット値を用いることで，適切なクラスタ数を定量的に決定することができる．

- シルエット値の計算
- あるデータ点と同クラスター内のデータ店とのユークリッド平方距離の平均　OCI
- あるデータ点と近隣クラスター内のデータ点とのユークリッド平方距離の平均　NCI

$$
\dfrac {NC_{i}-OC_{i}}{\max \left\{ NC_{i},OC_{i}\right\}}
$$

- 1に近いほど，よく当てはまってる

### 論文では

#### 手順
1. それぞれのデータ点のシルエット値を計算する
2. それぞれのクラスター数でのシルエット値の平均を出す
3. 最も高いシルエット平均を出したクラスター数を採用

#### その他

- ACNをTCN（閾値）で区別する
- 閾値は，使っていないFPGAのACNに近い値にする（平均？）
- 論文では決め方を近い値としか書いてなかった

- 論文内では，三つのLUTでの検出率を調べていた
- また，経年劣化日数での検出率の差を示していた（1day, 1week, 1month, 6month)

- FPGAの種類はSparten 3E だった
- このFPGAではACNの閾値を3以下を正常，3より上を異常と判断すると良いという結論

In [1]:
import numpy as np
import pandas as pd
import pickle
np.random.seed(100)

def kmeans(data, k):
    """kmeans++"""
    feature_num = data.shape[0]
    centers = np.zeros(k)
    distance = np.zeros(feature_num)
    probability = np.repeat(1/feature_num, feature_num)
    centers[0] = np.random.choice(data, 1, p=probability)
    for i in range(feature_num):
        distance[i] = np.abs(data[i] - centers[0]) ** 2

    for i in range(1, k):
        probability = []
        for j in range(feature_num):
            probability.append(distance[j]/np.sum(distance))

        centers[i] = np.random.choice(data, 1, p=probability)
        for j in range(feature_num):
            tmp_1 = float('inf')
            for k in range(i):
                tmp_2 = np.abs(data[j] - centers[k]) ** 2 
                if tmp_1 >= tmp_2:
                    tmp_1 = tmp_2
            distance[j] = tmp_1

    """kmeans"""
    data = np.array(data, dtype=float)
    index_list = np.zeros(len(data))
    centers = np.sort(np.array(centers, dtype=float))
    old_centers = np.zeros(len(centers))

    flag = True 
    while flag:
        # index割り当て
        for i, d in enumerate(data):
            min_d = float('inf')
            for j, c in enumerate(centers):
                dist = (abs(d - c))**2
                if min_d > dist:
                    min_d = dist
                    index_list[i] = j

        # center更新
        for i, c in enumerate(centers):
            m = 0; n = 0
            for j, il in enumerate(index_list):
                if il == i:
                    m += data[j]
                    n += 1
            if m != 0:
                m /= n
                old_centers[i] = c
                centers[i] = m

        # 収束チェック
        for i, c in enumerate(centers):
            if c != old_centers[i]:
                flag = True
                break
            else:
                flag = False

    return index_list, centers

def silhouette(data, index_list, center_num):
    SVI_list = []
    # データiのOCIとNCIから,SVIを求める
    for i in range(len(data)): # データ
        OCI = 0
        NCI = []
        # データiとの他のデータ点との距離を求める
        for j in range(center_num): # センターごとに求めていく
            tmp_list_1 = []
            for k in range(len(data)):  
                if index_list[k] == j:
                    tmp_list_1.append(data[k])
            tmp_list_2 = []
            for k in tmp_list_1:
                tmp_list_2.append(abs(data[i] - k)**2)

            if index_list[i] == j:
                OCI = np.sum(np.array(tmp_list_2)) / (len(tmp_list_2) - 1)
            else:
                NCI.append(np.mean(np.array(tmp_list_2)))

        NCI.sort()
        svi = (NCI[0] - OCI) / max(NCI[0], OCI)
        SVI_list.append(svi)

    return np.mean(np.array(SVI_list))


def main():
    data = []
    for i in range(1, 51):
        tmp_data = pd.read_csv('fresh_aged_ieice/s'+str(i)+'.csv', header=None).values
        data.append(tmp_data)
    for i in range(1, 3):
        tmp_data = pd.read_csv('fresh_aged_ieice/s'+str(i)+'_aged.csv', header=None).values
        data.append(tmp_data)
    data = np.array(data)

    check = []
    for i in range(148):
        for j in range(33):
            if data[0, i, j] == 0:
                check.append([i,j])
    for i in range(52):
        for j in range(148):
            for k in range(33):
                if [j,k] in check:
                    data[i, j, k] = 0

    tmp_1 = []
    for i in range(52):
        tmp_2 = data[i].flatten()
        tmp_1.append(tmp_2[tmp_2 != 0])
    data = np.array(tmp_1)

    SVI_list = []
    for i in range(52):
        svi_list = []
        for j in range(2,9):
            index_num, centers = kmeans(data[i], j)
            tmp = silhouette(data[i], index_num, len(centers))
            svi_list.append(tmp)
            print(j)
        SVI_list.append(svi_list)
        print(i+1)

    ACN_list = []
    ACN_index_list = []
    STATUS_list = []
    for i in range(52):
        tmp = np.argmax(SVI_list[i])
        ACN_list.append(SVI_list[i][tmp])
        ACN_index_list.append(tmp+2)
    for i in range(50):
        STATUS_list.append('unused')
    for i in range(2):
        STATUS_list.append('aged')

    table = [ACN_list, ACN_index_list, STATUS_list]
    table = np.array(table).T
    df = pd.DataFrame(table, columns=['Maximum Average Silhouette value', 'Appropriate Cluster Number', 'Status'], index=np.arange(1,53))
    print(df)
    
    f = open('ACN_table.binaryfile', 'wb')
    pickle.dump(table, f)
    f.close()


if __name__ == "__main__":
    main()
    pass

2
3
4
5
6
7
8
1
2
3
4
5
6
7
8
2
2
3
4
5
6
7
8
3
2
3
4
5
6
7
8
4
2
3
4
5
6
7
8
5
2
3
4
5
6
7
8
6
2
3
4
5
6
7
8
7
2
3
4
5
6
7
8
8
2
3
4
5
6
7
8
9
2
3
4
5
6
7
8
10
2
3
4
5
6
7
8
11
2
3
4
5
6
7
8
12
2
3
4
5
6
7
8
13
2
3
4
5
6
7
8
14
2
3
4
5
6
7
8
15
2
3
4
5
6
7
8
16
2
3
4
5
6
7
8
17
2
3
4
5
6
7
8
18
2
3
4
5
6
7
8
19
2
3
4
5
6
7
8
20
2
3
4
5
6
7
8
21
2
3
4
5
6
7
8
22
2
3
4
5
6
7
8
23
2
3
4
5
6
7
8
24
2
3
4
5
6
7
8
25
2
3
4
5
6
7
8
26
2
3
4
5
6
7
8
27
2
3
4
5
6
7
8
28
2
3
4
5
6
7
8
29
2
3
4
5
6
7
8
30
2
3
4
5
6
7
8
31
2
3
4
5
6
7
8
32
2
3
4
5
6
7
8
33
2
3
4
5
6
7
8
34
2
3
4
5
6
7
8
35
2
3
4
5
6
7
8
36
2
3
4
5
6
7
8
37
2
3
4
5
6
7
8
38
2
3
4
5
6
7
8
39
2
3
4
5
6
7
8
40
2
3
4
5
6
7
8
41
2
3
4
5
6
7
8
42
2
3
4
5
6
7
8
43
2
3
4
5
6
7
8
44
2
3
4
5
6
7
8
45
2
3
4
5
6
7
8
46
2
3
4
5
6
7
8
47
2
3
4
5
6
7
8
48
2
3
4
5
6
7
8
49
2
3
4
5
6
7
8
50
2
3
4
5
6
7
8
51
2
3
4
5
6
7
8
52
   Maximum Average Silhouette value Appropriate Cluster Number  Status
1                0.7042043758548217                   

# 実験結果

- 経年劣化したFPGAを見分けることができなかった
- シルエット値計算でわかった最適なクラスタ数は，ほぼ全て『２』という結果になった

## skleanのライブラリも使って実験してみる

In [3]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
np.random.seed(100)

data = []
for i in range(1, 51):
    tmp_data = pd.read_csv('fresh_aged_ieice/s'+str(i)+'.csv', header=None).values
    data.append(tmp_data)
for i in range(1, 3):
    tmp_data = pd.read_csv('fresh_aged_ieice/s'+str(i)+'_aged.csv', header=None).values
    data.append(tmp_data)
data = np.array(data)

check = []
for i in range(148):
    for j in range(33):
        if data[0, i, j] == 0:
            check.append([i,j])
for i in range(52):
    for j in range(148):
        for k in range(33):
            if [j,k] in check:
                data[i, j, k] = 0

tmp_1 = []
for i in range(52):
    tmp_2 = data[i].flatten()
    tmp_1.append(tmp_2[tmp_2 != 0])
data = np.array(tmp_1)

tmp_1 = []
for d in data:
    tmp_2 = []
    for i in range(len(d)):
        tmp_2.append([d[i], 0])
    tmp_1.append(tmp_2)
data = np.array(tmp_1)

SVI_list = []
for i in range(data.shape[0]):
    tmp = []
    for k in range(2, 9):
        km = KMeans(n_clusters=k,    # クラスターの個数
            init='k-means++',        # セントロイドの初期値をランダムに設定
            n_init=10,               # 異なるセントロイドの初期値を用いたk-meansアルゴリズムの実行回数
            max_iter=300,            # k-meansアルゴリズムの内部の最大イテレーション回数
            tol=1e-04,               # 収束と判定するための相対的な許容誤差
            random_state=100)          # セントロイドの初期化に用いる乱数発生器の状態
        model = km.fit_predict(data[i])
        score = silhouette_score(data[i], model)
        tmp.append(score)
        print(f'{k}回目')
    print(f'[{i+1}回目]')
    SVI_list.append(tmp)

index_list = np.arange(1,53)
acn_list = []
average_list = []
status_list = []
for svi in SVI_list:
    acn = np.argmax(svi) + 2
    average = svi[acn-2]
    acn_list.append(acn)
    average_list.append(average)
for i in range(50):
    status_list.append('unused')
for i in range(2):
    status_list.append('aged')

table = [average_list, acn_list, status_list]
table = np.array(table).T
df = pd.DataFrame(table,
                columns=['Maximum Average Silhouette value', 'Appropriate Cluster Number', 'Status'],
                index=index_list)
print(df)

2回目
3回目
4回目
5回目
6回目
7回目
8回目
[1回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[2回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[3回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[4回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[5回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[6回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[7回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[8回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[9回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[10回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[11回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[12回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[13回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[14回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[15回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[16回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[17回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[18回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[19回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[20回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[21回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[22回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[23回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[24回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[25回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[26回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[27回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[28回目]
2回目
3回目
4回目
5回目
6回目
7回目
8回目
[

# 実験結果

- 自作kmeansと変わりない結果となった
- 経年劣化FPGAはいずれも検出できなかった

- 論文内では，経年劣化日数を変化させ調査させていた
    - さらに経年させればシルエット値に変化がみられるかも？？